In [ ]:
import os
import time
import requests
import tarfile
import numpy as np
import argparse

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam


################################
###  GAT LAYER DEFINITION    ###
################################


class GraphAttentionLayer(nn.Module):
    """
    Graph Attention Layer (GAT) as described in the paper `"Graph Attention Networks" <https://arxiv.org/pdf/1710.10903.pdf>`.

        This operation can be mathematically described as:

            e_ij = a(W h_i, W h_j)
            α_ij = softmax_j(e_ij) = exp(e_ij) / Σ_k(exp(e_ik))
            h_i' = σ(Σ_j(α_ij W h_j))

            where h_i and h_j are the feature vectors of nodes i and j respectively, W is a learnable weight matrix,
            a is an attention mechanism that computes the attention coefficients e_ij, and σ is an activation function.

    """

    def __init__(
        self,
        in_features: int,
        out_features: int,
        n_heads: int,
        concat: bool = False,
        dropout: float = 0.4,
        leaky_relu_slope: float = 0.2,
    ):
        super(GraphAttentionLayer, self).__init__()

        self.n_heads = n_heads  # Number of attention heads
        self.concat = concat  # wether to concatenate the final attention heads
        self.dropout = dropout  # Dropout rate

        if concat:  # concatenating the attention heads
            self.out_features = out_features  # Number of output features per node
            assert out_features % n_heads == 0  # Ensure that out_features is a multiple of n_heads
            self.n_hidden = out_features // n_heads
        else:  # averaging output over the attention heads (Used in the main paper)
            self.n_hidden = out_features

        #  A shared linear transformation, parametrized by a weight matrix W is applied to every node
        #  Initialize the weight matrix W
        self.W = nn.Parameter(torch.empty(size=(in_features, self.n_hidden * n_heads)))

        # Initialize the attention weights a
        self.a = nn.Parameter(torch.empty(size=(n_heads, 2 * self.n_hidden, 1)))

        self.leakyrelu = nn.LeakyReLU(leaky_relu_slope)  # LeakyReLU activation function
        self.softmax = nn.Softmax(dim=1)  # softmax activation function to the attention coefficients

        self.reset_parameters()  # Reset the parameters

    def reset_parameters(self):
        """
        Reinitialize learnable parameters.
        """
        nn.init.xavier_normal_(self.W)
        nn.init.xavier_normal_(self.a)

    def _get_attention_scores(self, h_transformed: torch.Tensor):
        """calculates the attention scores e_ij for all pairs of nodes (i, j) in the graph
        in vectorized parallel form. for each pair of source and target nodes (i, j),
        the attention score e_ij is computed as follows:

            e_ij = LeakyReLU(a^T [Wh_i || Wh_j])

            where || denotes the concatenation operation, and a and W are the learnable parameters.

        Args:
            h_transformed (torch.Tensor): Transformed feature matrix with shape (n_nodes, n_heads, n_hidden),
                where n_nodes is the number of nodes and out_features is the number of output features per node.

        Returns:
            torch.Tensor: Attention score matrix with shape (n_heads, n_nodes, n_nodes), where n_nodes is the number of nodes.
        """

        source_scores = torch.matmul(h_transformed, self.a[:, : self.n_hidden, :])
        target_scores = torch.matmul(h_transformed, self.a[:, self.n_hidden :, :])

        # broadcast add
        # (n_heads, n_nodes, 1) + (n_heads, 1, n_nodes) = (n_heads, n_nodes, n_nodes)
        e = source_scores + target_scores.mT
        return self.leakyrelu(e)

    def forward(self, h: torch.Tensor, adj_mat: torch.Tensor):
        """
        Performs a graph attention layer operation.

        Args:
            h (torch.Tensor): Input tensor representing node features.
            adj_mat (torch.Tensor): Adjacency matrix representing graph structure.

        Returns:
            torch.Tensor: Output tensor after the graph convolution operation.
        """
        n_nodes = h.shape[0]

        # Apply linear transformation to node feature -> W h
        # output shape (n_nodes, n_hidden * n_heads)
        h_transformed = torch.mm(h, self.W)
        h_transformed = F.dropout(h_transformed, self.dropout, training=self.training)

        # splitting the heads by reshaping the tensor and putting heads dim first
        # output shape (n_heads, n_nodes, n_hidden)
        h_transformed = h_transformed.view(n_nodes, self.n_heads, self.n_hidden).permute(1, 0, 2)

        # getting the attention scores
        # output shape (n_heads, n_nodes, n_nodes)
        e = self._get_attention_scores(h_transformed)

        # Set the attention score for non-existent edges to -9e15 (MASKING NON-EXISTENT EDGES)
        connectivity_mask = -9e16 * torch.ones_like(e)
        e = torch.where(adj_mat > 0, e, connectivity_mask)  # masked attention scores

        # attention coefficients are computed as a softmax over the rows
        # for each column j in the attention score matrix e
        attention = F.softmax(e, dim=-1)
        attention = F.dropout(attention, self.dropout, training=self.training)

        # final node embeddings are computed as a weighted average of the features of its neighbors
        h_prime = torch.matmul(attention, h_transformed)

        # concatenating/averaging the attention heads
        # output shape (n_nodes, out_features)
        if self.concat:
            h_prime = h_prime.permute(1, 0, 2).contiguous().view(n_nodes, self.out_features)
        else:
            h_prime = h_prime.mean(dim=0)

        return h_prime

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class GraphAttentionLayer(nn.Module):
    """
    图注意力层

    注意力分数：e_ij = a(W h_i, W h_j)

    Args:
        nn (_type_): _description_
    """

    def __init__(
        self,
        in_features: int,
        out_features: int,
        n_heads: int,
        concat: bool = False,
        dropout: float = 0.4,
        leaky_relu_slope: float = 0.2,
    ):
        super(GraphAttentionLayer, self).__init__()

        self.n_heads = n_heads
        self.concat = concat
        self.dropout = dropout

        if concat:
            self.out_features = out_features
            assert out_features % n_heads == 0
            self.n_hidden = out_features // n_heads
        else:
            self.n_hidden = out_features

        self.W = nn.Parameter(torch.empty(size=(in_features, self.n_hidden * n_heads)))
        self.a = nn.Parameter(torch.empty(size=(n_heads, 2 * self.n_hidden, 1)))
        self.leakyrelu = nn.LeakyReLU(leaky_relu_slope)
        self.softmax = nn.Softmax(dim=1)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_normal_(self.W)
        nn.init.xavier_normal_(self.a)

    def _get_attention_scores(self, h_transformed: torch.Tensor):
        # 核心点是a记录了注意力得分，a[head, 2 * n_hidden, 1]表示第一个头中，把向量映射成注意力得分
        source_scores = torch.matmul(
            h_transformed, self.a[:, : self.n_hidden, :]
        )  # (n_heads, n_nodes, 1) 原节点注意力贡献
        target_scores = torch.matmul(
            h_transformed, self.a[:, self.n_hidden :, :]
        )  # (n_heads, n_nodes, 1) 目标节点注意力贡献
        e = source_scores + target_scores.permute(0, 2, 1)  # 广播相加
        return self.leakyrelu(e)

    def forward(self, h: torch.Tensor, adj_mat: torch.Tensor):
        n_nodes = h.shape[0]
        h_transformed = torch.mm(h, self.W)
        h_transformed = F.dropout(h_transformed, self.dropout, training=self.training)
        h_transformed = h_transformed.view(n_nodes, self.n_heads, self.n_hidden).permute(
            1, 0, 2
        )  # (n_heads, n_nodes, n_hidden)
        e = self._get_attention_scores(h_transformed)  # (n_heads, n_nodes, n_nodes)
        connectivity_mask = -9e15 * torch.ones_like(e)
        e = torch.where(adj_mat > 0, e, connectivity_mask)
        attention = F.softmax(e, dim=-1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, h_transformed)
        if self.concat:
            h_prime = h_prime.permute(1, 0, 2).contiguous().view(n_nodes, self.out_features)
        else:
            h_prime = h_prime.mean(dim=0)
        return h_prime